In [200]:
import pandas as pd
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt

from sklearn.neighbors import LocalOutlierFactor

In [201]:

# Ruta utilizando `/` en lugar de `\`
url = 'C:/Users/andre/OneDrive/Documentos/ciencia de datos/df_proyecto/df_proyecto_1.xlsx'
df = pd.read_excel(url)

### Campos Originales
- **orden**: Identificador único de la orden.
- **linea**: Número de línea dentro de la orden.
- **entrega**: Número de entrega asociada a la orden.
- **orden_larga**: Número largo o alternativo de la orden.
- **codigo**: Código único asociado al producto o artículo de la orden.
- **TNXDTE_15**: Fecha en que se procesó la orden.
- **pais**: País de origen del pedido.
- **CreationDate**: Fecha de creación de la orden.
- **fecha_aproximada**: Fecha estimada de entrega de la orden.
- **mes_aproximado**:  Mes estimado para la entrega.
- **nit_proveedor**: Número de Identificación Tributaria (NIT) del proveedor.
- **nombre_cliente**: Nombre del cliente que realizó la orden.
- **cantidad**: Cantidad de productos o unidades en la orden.
- **forma_envio**: Método de envío utilizado para la entrega.
- **moneda**: Moneda en la que se facturó la orden.
- **categoria**: Categoría del producto en la orden.
- **unidad_de_medida/U_codigo**: Unidad de medida utilizada para la cantidad del producto.
- **dias_diferencia_entrega**: Diferencia en días entre la fecha de entrega estimada y la fecha real de entrega.
- **cumplio**: Indicador de si la entrega cumplió con los tiempos estimados.


1.LIMPIEZA DE DATOS

1.1.datos generales data set

In [202]:
numero_de_datos=df.shape[0]
numero_de_features=df.shape[1]
print(f'el data set tiene {numero_de_datos} datos ')
print(f'el data set tiene {numero_de_features} variables ')

el data set tiene 9639 datos 
el data set tiene 21 variables 


In [203]:
#variables categóricas
df_cat = df.select_dtypes(include = ["object", 'category']).columns.tolist()
print(f'el data set tiene {len(df_cat)} variables categoricas')

el data set tiene 11 variables categoricas


In [204]:
df_num=df.select_dtypes(include = ['float64','float64','int32','int64']).columns.tolist()
print(f'el data set tiene {len(df_num)} variables numericas')

el data set tiene 6 variables numericas


In [205]:
df_fecha=df.select_dtypes(include = ['datetime64[ns]']).columns.tolist()
print(f'el data set tiene {len(df_fecha)} variables de fecha')


el data set tiene 4 variables de fecha


In [206]:
# valores unicos en las categorias 
for i in df_cat:
    print(f'{i}: {df[i].nunique()}')


codigo: 2843
pais: 27
dia_semana_aproximado: 7
mes_aproximado: 12
nit_proveedor: 283
nombre_proveedor: 281
forma_envio: 5
moneda: 4
categoria: 24
unidad_de_medida/U_codigo: 5
cumplio: 2


1.2.nullos

In [207]:
# Calcular la cantidad de valores nulos por cada columna en el DataFrame
nullos_features = df.isnull().sum()

# Calcular el total de valores nulos en todo el DataFrame sumando los nulos por cada columna
total_nullos = sum(df.isnull().sum())

# Mostrar el total de valores nulos
total_nullos


0

1.3 agragar caracteristicas y eliminar

In [208]:
print(f'la cantidad de duplicados es: {df.duplicated().sum()}')

la cantidad de duplicados es: 78


In [209]:
# Eliminar filas duplicadas del DataFrame
df.drop_duplicates(inplace=True)


In [210]:
# Eliminar columnas específicas del DataFrame NO necesarias
df.drop(['nit_proveedor', 'TNXDTE_15', 'linea', 'entrega', 'CreationDate', 'orden_larga', 'codigo'], axis=1, inplace=True)


In [211]:
# se agrega la caracteristica año 
df['año']=df.fecha_entrega_real.dt.year

In [212]:
# A la columna 'nombre_proveedor' del DataFrame df se le aplicará una transformación.
# La función str.strip() elimina los espacios en blanco que pudieran estar
# al inicio o al final de cada nombre en la columna 'nombre_proveedor'.

df['nombre_proveedor'] = df['nombre_proveedor'].str.strip()


1.3.1  outliers

In [213]:
# metodo zscore para eliminar outliers
media = df.dias_diferencia_entrega.mean()
desviacion_std = df.dias_diferencia_entrega.std()
z_score = ((df.dias_diferencia_entrega - media) / desviacion_std).abs()
outliers = df[z_score > 2]

# se filtra data set de outliers
df = df[z_score <= 2]


1.4 DATA SET CON CATEGORIAS PAISES 

In [214]:
# Se está creando una copia del DataFrame original 'df' y asignándola a una nueva variable 'df_cat_paises'.
# Esto asegura que cualquier modificación hecha a 'df_cat_paises' no afecte al DataFrame original 'df'.

df_cat_paises = df.copy()



In [215]:
categorias_geograficas = {
    # Asia
    'CHINA': 'Asia',
    'SOUTH KOREA': 'Asia',
    'TAIWAN': 'Asia',
    'HONG KONG': 'Asia',
    'SINGAPORE': 'Asia',
    'ASIA': 'Asia',

    # Norteamérica
    'ESTADOS UNIDOS': 'Norteamérica',
    'CANADA': 'Norteamérica',
    'MEXICO': 'Norteamérica',

    # Europa
    'ESPAÑA': 'Europa',
    'ITALIA': 'Europa',
    'ALEMANIA': 'Europa',
    'UNITED KINGDOM': 'Europa',
    'PORTUGAL': 'Europa',
    'FINLANDIA': 'Europa',
    'POLONIA': 'Europa',
    'AUSTRIA': 'Europa',
    'TURKEY': 'Europa',

    # América Latina
    'BRASIL': 'América Latina',
    'ARGENTINA': 'América Latina',
    'CHILE': 'América Latina',
    'PERU': 'América Latina',
    'ECUADOR': 'América Latina',
    'COSTA RICA': 'América Latina',
    'PANAMA': 'América Latina',

    # Oceanía
    'NEW ZELAND': 'Oceanía',

    # Oriente Medio
    'ISRAEL': 'Oriente Medio'
}

# Agregar columna de continente basada en el mapeo
df_cat_paises['Continente'] = df_cat_paises['pais'].map(categorias_geograficas)

#df.drop('pais', axis=1, inplace=True)
df_cat_paises.drop(['nombre_proveedor','pais'], axis=1, inplace=True)


In [216]:
# Se guarda el DataFrame 'df_cat_paises' en un archivo CSV.
# El archivo se guardará en la ruta especificada: 'C:/Users/andre/OneDrive/Documentos/ciencia de datos/df_proyecto/data_c_paises.csv'.
# La opción 'index=False' asegura que no se incluya el índice del DataFrame como una columna adicional en el archivo CSV.

df_cat_paises.to_csv('C:/Users/andre/OneDrive/Documentos/ciencia de datos/df_proyecto/data_c_paises.csv', index=False)



1.4.2 DATA SET CATEGORIA PROVEEDORES

In [217]:
df_cat_prov=df.copy()

In [218]:
# este codigo calcula el numero de ordenes por proveedor y los años de proveedor de la empresa
# esto con el fin sacar el promedio anual de ordenes de provision por proveedor

tabla_proveedores=df_cat_prov.groupby('nombre_proveedor').agg(total_conteo=('nombre_proveedor', 'count')\
                                        ,años_de_ventas=('año', 'nunique')).sort_values(by='total_conteo', ascending=False).reset_index()

tabla_proveedores['promedio_anual']=round(tabla_proveedores.total_conteo/tabla_proveedores.años_de_ventas,0)

In [219]:
def categorizar_proveedor(row):
    # Proveedor Estratégico: Más de 3 años y más de 60 ORDENES en promedio anual
    if row['años_de_ventas'] > 3 and row['promedio_anual'] >= 30:
        return 'Proveedor Estratégico'
    # Proveedor Confiable: Más de 2 años y entre 40 y 60 ORDENES en promedio anual
    elif row['años_de_ventas'] > 2 and 20 <= row['promedio_anual'] < 30:
        return 'Proveedor Confiable'
    # Proveedor Regular: Al menos 1 año y entre 15 y 40 ORDENES en promedio anual
    elif row['años_de_ventas'] >= 1 and 3 <= row['promedio_anual'] < 20:
        return 'Proveedor Regular'
    # Proveedor Ocasional: Al menos 1 año y entre 3 y 15 ORDENES en promedio anual
    elif row['años_de_ventas'] >= 1 and 1 <= row['promedio_anual'] < 3:
        return 'Proveedor Ocasional'
    # Proveedor Esporádico: Menos de 1 año o promedio anual menor a 3
    else:
        return 'Proveedor Esporádico'



In [220]:
# Se crea una nueva columna 'cate_proveedor' en el DataFrame 'tabla_proveedores'.
# La función 'categorizar_proveedor' se aplica a cada fila del DataFrame usando 'apply'.
tabla_proveedores['cate_proveedor'] = tabla_proveedores.apply(categorizar_proveedor, axis=1)

In [221]:
# Se realiza una fusión (merge) de los DataFrames 'df_cat_prov' y 'tabla_proveedores' en base a la columna 'nombre_proveedor'.
# La opción 'how="inner"' indica que solo se mantendrán las filas que tengan coincidencias en ambas tablas (fusión interna).
# Esto garantiza que solo los proveedores presentes en ambos DataFrames aparecerán en el DataFrame resultante.

df_cat_prov = pd.merge(df_cat_prov, tabla_proveedores, on='nombre_proveedor', how='inner')


In [222]:
df_cat_prov=df_cat_prov[['orden','fecha_aproximada','dia_semana_aproximado','mes_aproximado','fecha_entrega_real','cantidad',\
                         'forma_envio','moneda','categoria','unidad_de_medida/U_codigo','dias_diferencia_entrega','cumplio','año','cate_proveedor','pais']]


In [223]:
df_cat_prov.to_csv('C:/Users/andre/OneDrive/Documentos/ciencia de datos/df_proyecto/data_cat_prov.csv', index=False)

1.4.3 DATA SET CATEGORIA PROVEEDORES y PAIS

In [224]:
df_c_=df_cat_prov.copy()

In [225]:
df_c_['Continente'] = df_c_['pais'].map(categorias_geograficas)

In [226]:
df_c_.drop('pais', axis=1, inplace=True)

In [227]:
df_c_.to_csv('C:/Users/andre/OneDrive/Documentos/ciencia de datos/df_proyecto/data_cat_total.csv', index=False)

1.4.4 DATA SET PROVEEDORES INCUMPLEN

In [242]:
df_prove_incum=df_c_[df_c_.cumplio=='NO CUMPLIO']

In [243]:
df_prove_incum.to_csv('C:/Users/andre/OneDrive/Documentos/ciencia de datos/df_proyecto/data_prove_incum.csv', index=False)